##### Copyright 2021 The TensorFlow Authors.

The Model Maker library uses *transfer learning* to simplify the process of training a TensorFlow Lite model using a custom dataset. Retraining a TensorFlow Lite model with your own custom dataset reduces the amount of training data required and will shorten the training time.


This tutorial uses the EfficientDet-Lite0 model. EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture.

## Prerequisites

Trust the 'requirements.txt' file to install the correct versions of the packages, and install them in one go :

In [3]:
# !pip install -r requirements.txt
# !pip install roboflow
# !pip uninstall -y tensorflow && pip install -q tensorflow==2.8.0

Found existing installation: tensorflow 2.8.4
Uninstalling tensorflow-2.8.4:
  Successfully uninstalled tensorflow-2.8.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflowjs 3.18.0 requires packaging~=20.9, but you have packaging 23.2 which is incompatible.
tflite-model-maker 0.4.2 requires matplotlib<3.5.0,>=3.0.3, but you have matplotlib 3.8.2 which is incompatible.
tflite-model-maker 0.4.2 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 2.1.0 which is incompatible.


Import the required packages. Those marked as unused are still necessary to make the Model Maker library work.

In [5]:
import numpy as np
import pandas as pd
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
import pycocotools as pycocotools

import tensorflow as tf
assert tf.__version__.startswith('2')
MODEL_NAME = "efficientdet_lite4"
tf.get_logger().setLevel('ERROR')

from absl import logging
logging.set_verbosity(logging.ERROR)

### Prepare the dataset

We need to have a dataset cleaned and ready to be used.

The dataset is provided in CSV format:
```
TRAINING,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Salad,0.0,0.0954,,,0.977,0.957,,
VALIDATION,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Seafood,0.0154,0.1538,,,1.0,0.802,,
TEST,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Tomato,0.0,0.655,,,0.231,0.839,,
```

* Each row corresponds to an object localized inside a larger image, with each object specifically designated as test, train, or validation data. You'll learn more about what that means in a later stage in this notebook.
* The three lines included here indicate **three distinct objects located inside the same image** available at `gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg`.
* Each row has a different label: `Salad`, `Seafood`, `Tomato`, etc.
* Bounding boxes are specified for each image using the top left and bottom right vertices.

If you want to know more about how to prepare your own CSV file and the minimum requirements for creating a valid dataset, see the [Preparing your training data](https://cloud.google.com/vision/automl/object-detection/docs/prepare) guide for more details.

If you are new to Google Cloud, you may wonder what the `gs://` URL means. They are URLs of files stored on [Google Cloud Storage](https://cloud.google.com/storage) (GCS). If you make your files on GCS public or [authenticate your client](https://cloud.google.com/storage/docs/authentication#libauth), Model Maker can read those files similarly to your local files.

However, you don't need to keep your images on Google Cloud to use Model Maker. You can use a local path in your CSV file and Model Maker will just work.

## Quickstart

In [6]:
spec = model_spec.get(MODEL_NAME)

2024-01-06 11:01:15.910482: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/theojeannes/anaconda3/envs/tflite-model-maker/lib/python3.9/site-packages/cv2/../../lib64:
2024-01-06 11:01:15.910522: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-01-06 11:01:15.910556: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (theojeannes): /proc/driver/nvidia/version does not exist
2024-01-06 11:01:15.913584: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler fla

Import**Step 2. Load the dataset.**

Model Maker will take input data in the CSV format. Use the `object_detector.DataLoader.from_csv` method to load the dataset and split them into the training, validation and test images.

* Training images: These images are used to train the object detection model to recognize salad ingredients.
* Validation images: These are images that the model didn't see during the training process. You'll use them to decide when you should stop the training, to avoid [overfitting](https://en.wikipedia.org/wiki/Overfitting).
* Test images: These images are used to evaluate the final model performance.

You can load the CSV file directly from Google Cloud Storage, but you don't need to keep your images on Google Cloud to use Model Maker. You can specify a local CSV file on your computer, and Model Maker will work just fine.

In [38]:
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('gs://cloud-ml-data/img/openimage/csv/salads_ml_use.csv')

**Step 3. Train the TensorFlow model with the training data.**

* `epochs = 50` by default, which means it will go through the training dataset 50 times. You can look at the validation accuracy during training and stop early to avoid overfitting.
* Set `batch_size = 8`, size of one batch (how many rough the 175 images in the training dataset.
* Set `train_whole_model=True` to fine-tune the whole model instead of just training the head layer to improve accuracy. The trade-off is that it may take longer to train the model.

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=8, train_whole_model=True, validation_data=validation_data,epochs=3)

Epoch 1/50


2024-01-05 15:58:51.295428: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


21/21 [==============================] - ETA: 0s - det_loss: 1.7212 - cls_loss: 1.1397 - box_loss: 0.0116 - reg_l2_loss: 0.1083 - loss: 1.8295 - learning_rate: 0.0090 - gradient_norm: 0.5891  

2024-01-05 16:25:16.104220: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


21/21 [==============================] - 1607s 74s/step - det_loss: 1.7168 - cls_loss: 1.1391 - box_loss: 0.0116 - reg_l2_loss: 0.1083 - loss: 1.8251 - learning_rate: 0.0090 - gradient_norm: 0.5937 - val_det_loss: 1.6749 - val_cls_loss: 1.1234 - val_box_loss: 0.0110 - val_reg_l2_loss: 0.1083 - val_loss: 1.7833
Epoch 2/50
21/21 [==============================] - ETA: 0s - det_loss: 1.5645 - cls_loss: 1.0955 - box_loss: 0.0094 - reg_l2_loss: 0.1083 - loss: 1.6728 - learning_rate: 0.0100 - gradient_norm: 0.9546  

...

21/21 [==============================] - 1780s 85s/step - det_loss: 0.7205 - cls_loss: 0.5200 - box_loss: 0.0040 - reg_l2_loss: 0.1085 - loss: 0.8290 - learning_rate: 0.0091 - gradient_norm: 2.4904 - val_det_loss: 0.9199 - val_cls_loss: 0.6067 - val_box_loss: 0.0063 - val_reg_l2_loss: 0.1085 - val_loss: 1.0283
Epoch 11/50
21/21 [==============================] - ETA: 0s - det_loss: 0.6900 - cls_loss: 0.4977 - box_loss: 0.0038 - reg_l2_loss: 0.1085 - loss: 0.7984 - learning_rate: 0.0089 - gradient_norm: 2.6997  

2024-01-05 21:09:15.699115: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


21/21 [==============================] - 1807s 86s/step - det_loss: 0.6955 - cls_loss: 0.5018 - box_loss: 0.0039 - reg_l2_loss: 0.1085 - loss: 0.8040 - learning_rate: 0.0089 - gradient_norm: 2.7983 - val_det_loss: 0.9187 - val_cls_loss: 0.6163 - val_box_loss: 0.0060 - val_reg_l2_loss: 0.1085 - val_loss: 1.0271
Epoch 12/50
21/21 [==============================] - ETA: 0s - det_loss: 0.6525 - cls_loss: 0.4691 - box_loss: 0.0037 - reg_l2_loss: 0.1085 - loss: 0.7610 - learning_rate: 0.0087 - gradient_norm: 2.7696  

2024-01-05 21:40:01.998898: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


21/21 [==============================] - 1820s 86s/step - det_loss: 0.6488 - cls_loss: 0.4668 - box_loss: 0.0036 - reg_l2_loss: 0.1085 - loss: 0.7572 - learning_rate: 0.0087 - gradient_norm: 2.7693 - val_det_loss: 0.9291 - val_cls_loss: 0.6129 - val_box_loss: 0.0063 - val_reg_l2_loss: 0.1085 - val_loss: 1.0376
Epoch 13/50
 7/21 [=========>....................] - ETA: 19:48 - det_loss: 0.6753 - cls_loss: 0.4803 - box_loss: 0.0039 - reg_l2_loss: 0.1085 - loss: 0.7838 - learning_rate: 0.0086 - gradient_norm: 2.6146

**Step 4. Evaluate the model with the test data.**

We are mainly interested in the AP75 (which is the mAP with IoU threshold 0.75) and ARmax10(Average Recall, and we use the top 10 confidence threshold to say if a result is a true positive or a false positive). The higher the better. Here we are evaluating on the model

In [ ]:
metrics = ['AP75','APs','APm','APl','ARmax10','ARs','ARm','ARl']

In [ ]:
res = model.evaluate(test_data)
data = {m: res[m] for m in metrics}
df = pd.DataFrame(list(data.items()), columns=['Metric', 'Vanilla'])

# Set 'Metric' as the index
df.set_index('Metric', inplace=True)
df

**Step 5.  Export as a TensorFlow Lite model.**

Export the trained object detection model to the TensorFlow Lite format by specifying which folder you want to export the quantized model to. The default post-training quantization technique is full integer quantization.

Several factors can affect the model accuracy when exporting to TFLite:
* [Quantization](https://www.tensorflow.org/lite/performance/model_optimization) helps shrinking the model size by 4 times at the expense of some accuracy drop.
* The original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate.
Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.

Therefore, you'll have to evaluate the exported TFLite model and compare its accuracy with the original TensorFlow model.

In [ ]:
quantization_configs = {
    'vanilla': None,
    'float16': QuantizationConfig.for_float16(),
    'float32': QuantizationConfig.for_float32(),
}

In [ ]:
for k,v in quantization_configs.items():
    config_name = MODEL_NAME+"_"+k+".tflite"
    model.export(export_dir='.',tflite_filename=config_name ,quantization_config=v)
    res[k] = model.evaluate_tflite(config_name, test_data)
    df[k] = res[k][metrics].values()

In [ ]:
df

## Test the TFLite model on your image

Use the app

### Load the dataset

#### Load your own data

You can upload your own dataset to work through this tutorial. Upload your dataset by using the left sidebar in Colab.

<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_object_detection.png" alt="Upload File" width="1000" hspace="0">

If you prefer not to upload your dataset to the cloud, you can also locally run the library by following the [guide](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

#### Load your data with a different data format

The Model Maker library also supports the `object_detector.DataLoader.from_pascal_voc` method to load data with [PASCAL VOC](https://towardsdatascience.com/coco-data-format-for-object-detection-a4c5eaf518c5#:~:text=Pascal%20VOC%20is%20an%20XML,for%20training%2C%20testing%20and%20validation) format. [makesense.ai](https://www.makesense.ai/) and [LabelImg](https://github.com/tzutalin/labelImg) are the tools that can annotate the image and save annotations as XML files in PASCAL VOC data format:
```python
object_detector.DataLoader.from_pascal_voc(image_dir, annotations_dir, label_map={1: "person", 2: "notperson"})
```


### Customize the EfficientDet model hyperparameters

The model and training pipeline parameters you can adjust are:

* `model_dir`: The location to save the model checkpoint files. If not set, a temporary directory will be used.
* `steps_per_execution`: Number of steps per training execution.
* `moving_average_decay`: Float. The decay to use for maintaining moving averages of the trained parameters.
* `var_freeze_expr`: The regular expression to map the prefix name of variables to be frozen which means remaining the same during training. More specific, use `re.match(var_freeze_expr, variable_name)` in the codebase to map the variables to be frozen.
* `tflite_max_detections`: integer, 25 by default. The max number of output detections in the TFLite model.
* `strategy`:  A string specifying which distribution strategy to use. Accepted values are 'tpu', 'gpus', None. tpu' means to use TPUStrategy. 'gpus' mean to use MirroredStrategy for multi-gpus. If None, use TF default with OneDeviceStrategy.
* `tpu`:  The Cloud TPU to use for training. This should be either the name used when creating the Cloud TPU, or a grpc://ip.address.of.tpu:8470 url.
* `use_xla`: Use XLA even if strategy is not tpu. If strategy is tpu, always use XLA, and this flag has no effect.
* `profile`: Enable profile mode.
* `debug`: Enable debug mode.

Other parameters that can be adjusted is shown in [hparams_config.py](https://github.com/google/automl/blob/df451765d467c5ed78bbdfd632810bc1014b123e/efficientdet/hparams_config.py#L170).


For instance, you can set the `var_freeze_expr='efficientnet'` which freezes the variables with name prefix `efficientnet` (default is `'(efficientnet|fpn_cells|resample_p6)'`). This allows the model to freeze untrainable variables and keep their value the same through training.

```python
spec = model_spec.get('efficientdet_lite0')
spec.config.var_freeze_expr = 'efficientnet'
```

### Change the Model Architecture

You can change the model architecture by changing the `model_spec`. For instance, change the `model_spec` to the EfficientDet-Lite4 model.

```python
spec = model_spec.get('efficientdet_lite4')
```

### Tune the training hyperparameters

The `create` function is the driver function that the Model Maker library uses to create models. The `model_spec` parameter defines the model specification. The `object_detector.EfficientDetSpec` class is currently supported. The `create` function comprises of the following steps:

1. Creates the model for the object detection according to `model_spec`.
2. Trains the model.  The default epochs and the default batch size are set by the `epochs` and `batch_size` variables in the `model_spec` object.
You can also tune the training hyperparameters like `epochs` and `batch_size` that affect the model accuracy. For instance,

*   `epochs`: Integer, 50 by default. More epochs could achieve better accuracy, but may lead to overfitting.
*   `batch_size`: Integer, 64 by default. The number of samples to use in one training step.
*   `train_whole_model`: Boolean, False by default. If true, train the whole model. Otherwise, only train the layers that do not match `var_freeze_expr`.

For example, you can train with less epochs and only the head layer. You can increase the number of epochs for better results.

```python
model = object_detector.create(train_data, model_spec=spec, epochs=10, validation_data=validation_data)
```

### Export to different formats

The export formats can be one or a list of the following:

*   `ExportFormat.TFLITE`
*   `ExportFormat.LABEL`
*   `ExportFormat.SAVED_MODEL`




By default, it exports only the TensorFlow Lite model file containing the model [metadata](https://www.tensorflow.org/lite/models/convert/metadata) so that you can later use in an on-device ML application. The label file is embedded in metadata.

In many on-device ML application, the model size is an important factor. Therefore, it is recommended that you quantize the model to make it smaller and potentially run faster. As for EfficientDet-Lite models, full integer quantization  is used to quantize the model by default. Please refer to [Post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization) for more detail.

```python
model.export(export_dir='.')
```

You can also choose to export other files related to the model for better examination. For instance, exporting both the saved model and the label file as follows:
```python
model.export(export_dir='.', export_format=[ExportFormat.SAVED_MODEL, ExportFormat.LABEL])
```

### Customize Post-training quantization on the TensorFlow Lite model

[Post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization) is a conversion technique that can reduce model size and inference latency, while also improving CPU and hardware accelerator inference speed, with a little degradation in model accuracy. Thus, it's widely used to optimize the model.

Model Maker library applies a default post-training quantization technique when exporting the model. If you want to customize post-training quantization, Model Maker supports multiple post-training quantization options using [QuantizationConfig](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/config/QuantizationConfig) as well. Let's take float16 quantization as an instance. First, define the quantization config.

```python
config = QuantizationConfig.for_float16()
```


Then we export the TensorFlow Lite model with such configuration.

```python
model.export(export_dir='.', tflite_filename='model_fp16.tflite', quantization_config=config)
```

# Read more

You can read our [object detection](https://www.tensorflow.org/lite/examples/object_detection/overview) example to learn technical details. For more information, please refer to:

*   TensorFlow Lite Model Maker [guide](https://www.tensorflow.org/lite/models/modify/model_maker) and [API reference](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker).
*   Task Library: [ObjectDetector](https://www.tensorflow.org/lite/inference_with_metadata/task_library/object_detector) for deployment.
*   The end-to-end reference apps: [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/android), [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/ios), and [Raspberry PI](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/raspberry_pi).

